In [1]:
import sys
sys.path.append('../component')  # Ensure sys is imported before using it
import pandas as pd
from utils import *
from class_ARIMA_model import ARIMA_model
import os
os.getcwd()

'/home/ubuntu/Capstone_Group_1/code/main_code'

In [2]:
df = pd.read_csv('../data/weather.csv', parse_dates=['date'])
item_list = df.columns[1]
# plt_rolling_mean_var(df, item_list)
new_ts = differencing(df, 1, item_list)
df['1st_diff'] = new_ts
# plt_rolling_mean_var(df, df.columns[2])
# GPAC_table(df.iloc[:, 2], J=12, K=12)

In [3]:
df = pd.read_csv('../data/weather.csv', parse_dates=['date'])
item_list = df.columns[1]
new_ts = differencing(df, 1, item_list)
df['1st_diff'] = new_ts

In [4]:
df_train, df_temp = train_test_split(df, test_size=0.3, shuffle=False)
df_test, df_val = train_test_split(df_temp, test_size=0.5, shuffle=False)

df_train['date'] = pd.to_datetime(df_train['date'])
df_train.set_index('date', inplace=True)

df_test['date'] = pd.to_datetime(df_test['date'])
df_test.set_index('date', inplace=True)

data = df_train['1st_diff'].values.reshape(-1, 1)
print(data.shape)

(73684, 1)


# test test   

In [16]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(data, order=(1, 0, 0)).fit(method_kwargs = {
    'epsilon': 1e-8,     
    'maxfun': 500,  
})


In [6]:
# test  test   test
AR_model_grid = ARIMA_model(AR_order=1)
AR_model_grid.fit(data)

ARIMA_model()

# test test

In [5]:
# try the grid search for ARMA
param_grid = {'AR_order': range(1, 5),
              'MA_order': range(1,5)}  
model = ARIMA_model()

best_order = grid_sklearn(data, param_grid, model, n_splits=5)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting

KeyboardInterrupt: 

In [ ]:
# Grid search for AR model order
param_grid = {'AR_order': range(1, 11)}  
model = ARIMA_model()

best_order = grid_sklearn(data, param_grid, model, n_splits=5)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
AR_model_grid = ARIMA_model(AR_order=best_order)
AR_model_grid.fit(data)

# prediction
prediction = AR_model_grid.predict(data)

# forecast
forecast = AR_model_grid.forecast(df_test.shape[0])

In [ ]:
# reverse prediction
value = df_train.iloc[0, 0]
rev_predition = rev_diff(value, prediction)

# reverse forecast 
value = df_train.iloc[-1, 0]
rev_forecast = rev_diff(value, forecast)

In [ ]:
# MSE: test vs. forecast
MSE(df_test.iloc[:, 0], rev_forecast)

In [ ]:
plt_prediction(df_train, rev_predition)

In [ ]:
plt_forecast(df_test, rev_forecast)

In [ ]:
plt_train_test_forecast(df_train, df_test, rev_forecast)

### Optuna for Grid search

In [14]:
class ARIMA_model(BaseEstimator):
    def __init__(self, AR_order=1, MA_order=0, Inte_order=0):
        self.AR_order = AR_order
        self.MA_order = MA_order
        self.Inte_order = Inte_order
        self.model = None

    def fit(self, X, y=None):
        # self.model = ARIMA(X, order=(self.AR_order, self.Inte_order, self.MA_order)).fit(method_kwargs = {'epsilon': 1e-12, 'maxfun':500})
        self.model = ARIMA(X, order=(self.AR_order, self.Inte_order, self.MA_order)).fit(method='statespace', method_kwargs ={'maxiter':10})
        return self

    def predict(self, X):
        return self.model.predict(start=0, end=len(X)-1)

    def forecast(self, steps):
        return self.model.forecast(steps=steps)

In [15]:
study, best_order = optuna_search_ARIMA(data, ar_max=11, ma_max=11, n_trials=3)   # method_kwargs = {'epsilon': 1e-10, 'maxfun':500}

[I 2024-10-15 23:36:12,940] A new study created in memory with name: no-name-041816d3-dfd6-42dc-aa65-6940c58bc7bb
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximu

Best parameters: {'AR_order': 4, 'MA_order': 9}
Best MSE: 0.21967837437097848


In [9]:
study, best_order = optuna_search_ARIMA(data, ar_max=11, ma_max=11, n_trials=3)  # method_kwargs = {'epsilon': 1e-12, 'maxfun':500}

[I 2024-10-15 22:55:56,790] A new study created in memory with name: no-name-a39c3bcb-4aa1-483f-a654-d20ca07305c0
[I 2024-10-15 23:01:28,955] Trial 0 finished with value: 0.21988990427714317 and parameters: {'AR_order': 11, 'MA_order': 2}. Best is trial 0 with value: 0.21988990427714317.
[I 2024-10-15 23:06:03,983] Trial 1 finished with value: 0.21988989557959057 and parameters: {'AR_order': 10, 'MA_order': 2}. Best is trial 1 with value: 0.21988989557959057.
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ubuntu/.local/lib/python3.12/site-packages/statsmodels/base/model.py

Best parameters: {'AR_order': 3, 'MA_order': 2}
Best MSE: 0.21928364902856745


In [ ]:
AR_model_optuna = ARIMA_model(AR_order=best_order, MA_order=0, Inte_order=0)
AR_model_optuna.fit(data)

# prediction
prediction = AR_model_optuna.predict(start=0, end=df_train.shape[0]-1)
# forecast
forecast = AR_model_optuna.forecast(df_test.shape[0])

In [ ]:
# reverse prediction
value = df_train.iloc[0, 0]
rev_predition = rev_diff(value, prediction)

# reverse forecast 
value = df_train.iloc[-1, 0]
rev_forecast = rev_diff(value, forecast)

In [ ]:
# MSE: test vs. forecast
MSE(df_test.iloc[:, 0], rev_forecast)

In [ ]:
plt_prediction(df_train, rev_predition)

In [ ]:
plt_forecast(df_test, rev_forecast)

In [ ]:
plt_train_test_forecast(df_train, df_test, rev_forecast)

In [ ]:
# Plot the optimization history
plt.figure(figsize=(10, 6))
optuna.visualization.plot_optimization_history(study)
plt.show()

# Plot the parameter importances
plt.figure(figsize=(10, 6))
optuna.visualization.plot_param_importances(study)
plt.show()